In [3]:
import pandas as pd
import numpy as np
from pyproj import Proj, transform
df = pd.read_excel(r'../dataset/2023 Pollution Inventory Dataset V2.xlsx',skiprows=9)
df22 = pd.read_excel(r'../dataset/2022 Pollution Inventory Dataset V2.xlsx',skiprows=9)
df21 = pd.read_excel(r'../dataset/2021 Pollution Inventory Dataset.xlsx',skiprows=9)
df20 = pd.read_excel(r'../dataset/2020 Pollution Inventory Dataset V2.xlsx',skiprows=9)
df19 = pd.read_excel(r'../dataset/2019 Pollution Inventory Dataset V2.xlsx',skiprows=9)
df18 = pd.read_excel(r'../dataset/2018 Pollution Inventory.xlsx',skiprows=9)
df17 = pd.read_excel(r'../dataset/2017 Pollution Inventory Dataset.xlsx',skiprows=9)
df16 = pd.read_excel(r'../dataset/2016 Pollution Inventory dataset - version 2.xlsx', sheet_name=1, skiprows=9)

,AUTHORISATION ID / PERMIT ID,ACTIVITY DESCRIPTION,OPERATOR NAME,SITE ADDRESS,SITE POSTCODE,EASTING,NORTHING,EA AREA NAME,ROUTE NAME,SUBSTANCE NAME,REPORTING THRESHOLD (kg),QUANTITY RELEASED (kg),REGULATED INDUSTRY SECTOR,REGULATED INDUSTRY SUB SECTOR
0,UP3133UK,"INTENSIVE FARMING; > 40,000 POULTRY",Fridays Ltd,Chequer Tree Farm And Coursehorn Farm Poultry ...,TN17 3PN,578630.0,134230.0,"Kent, South London and East Sussex",Air,Ammonia,1000.0,0,Agriculture,Intensive Farming
1,QP3133UM,"INTENSIVE FARMING; > 40,000 POULTRY",Hook2sisters Ltd,Winfarthing Farm Poultry Unit Short Green Winf...,IP22 2EE,610040.0,286300.0,East Anglia,Air,Ammonia,1000.0,0,Agriculture,Intensive Farming
2,UP3133UK,"INTENSIVE FARMING; > 40,000 POULTRY",Fridays Ltd,Chequer Tree Farm And Coursehorn Farm Poultry ...,TN17 3PN,578630.0,134230.0,"Kent, South London and East Sussex",Air,Particulate matter - PM10,1000.0,0,Agriculture,Intensive Farming
3,QP3133UM,"INTENSIVE FARMING; > 40,000 POULTRY",Hook2sisters Ltd,Winfarthing Farm Poultry Unit Short Green Winf...,IP22 2EE,610040.0,286300.0,East Anglia,Air,Particulate matter - PM10,1000.0,0,Agriculture,Intensive Farming
4,AP3833US,"INTENSIVE FARMING; > 40,000 POULTRY",Hook2sisters Ltd,Martham East Farm Poultry Unit Rollesby Road M...,NR29 4SL,645750.0,317400.0,East Anglia,Air,Ammonia,1000.0,0,Agriculture,Intensive Farming


In [9]:
df16.head()

,AUTHORISATION ID / PERMIT ID,PRIMARY INDUSTRY ACTIVITY DESCRIPTION,OPERATOR NAME,SITE ADDRESS,SITE POSTCODE,EASTING,NORTHING,EA AREA NAME,ROUTE NAME,SUBSTANCE NAME,REPORTING THRESHOLD (kg),QUANTITY RELEASED (kg),REGULATED INDUSTRY SECTOR,REGULATED INDUSTRY SUB SECTOR
0,BJ5953IT,SURFACE TREATING METALS AND PLASTICS; ELECTROL...,Hyde Coatings Limited,HYDE COATINGS LIMITED BRIDGE HOUSE BRIDGE STRE...,SK15 1PH,395760.0,398270.0,Greater Manchester Merseyside and Cheshire,Air,Non-methane volatile organic compounds (NMVOCs),10000.00000,0,Metals,Surface treatment
1,BJ5953IT,SURFACE TREATING METALS AND PLASTICS; ELECTROL...,Hyde Coatings Limited,HYDE COATINGS LIMITED BRIDGE HOUSE BRIDGE STRE...,SK15 1PH,395760.0,398270.0,Greater Manchester Merseyside and Cheshire,Air,Particulate matter - total,10000.00000,0,Metals,Surface treatment
2,BL7701IS,NON-FERROUS METALS; PRODUCING ETC CADMIUM/MERC...,Fusion Automation Inc,The Pinnacles Barrows Road Harlow Essex,CM19 5AP,542700.0,209600.0,Hertfordshire and North London,Air,Cadmium,1.00000,0,Metals,Non-Ferrous
3,BL7701IS,NON-FERROUS METALS; PRODUCING ETC CADMIUM/MERC...,Fusion Automation Inc,The Pinnacles Barrows Road Harlow Essex,CM19 5AP,542700.0,209600.0,Hertfordshire and North London,Air,Lead,100.00000,0,Metals,Non-Ferrous
4,AP3835SM,"INCINERATION, OTHER THAN IN COURSE OF BURNING ...",MES Environmental Limited,Waste To Energy Plant Crown Street Wolverhampt...,WV1 1QB,391660.0,299990.0,West Midlands,Air,Polychlorinated biphenyls (PCBs) - as WHO TEQ,0.00001,0.0,Combustion,EfW


In [7]:
#dummy variables for certain columns
categorical_columns = ["EA AREA NAME", "ROUTE NAME", "REGULATED INDUSTRY SECTOR", "REGULATED INDUSTRY SUB SECTOR"]

for col in categorical_columns:
    unique_values = {value: idx for idx, value in enumerate(sorted(df[col].unique()))}
    df[col + " LABEL"] = df[col].map(unique_values)

#gases to be removed
remove_gases = [
    "Sulphur oxides (SO2 and SO3) as SO2", "Carbon monoxide", "Carbon dioxide", 
    "Carbon Dioxide From Qualifying Renewable Fuel Sources", "Methane", 
    "Nitrogen - as total N", "Nitrogen oxides (NO and NO2) as NO2", "Nitrous oxide", 
    "Sulphur hexafluoride", "Hydrogen cyanide", "Fluorine and inorganic fluorine compounds - as HF", 
    "Chlorofluorocarbons (CFCs)", "Hydrochlorofluorocarbons (HCFCs)", "Hydrofluorocarbons (HFCs)", 
    "Perfluorocarbons (PFCs)"]
#removing from dataset
df_no_gas = df[~df["SUBSTANCE NAME"].isin(remove_gases)]
df22_no_gas = df22[~df22["SUBSTANCE NAME"].isin(remove_gases)]
df21_no_gas = df21[~df21["SUBSTANCE NAME"].isin(remove_gases)]
df20_no_gas = df20[~df20["SUBSTANCE NAME"].isin(remove_gases)]
df19_no_gas = df19[~df19["SUBSTANCE NAME"].isin(remove_gases)]
df18_no_gas = df18[~df18["SUBSTANCE NAME"].isin(remove_gases)]
df17_no_gas = df17[~df17["SUBSTANCE NAME"].isin(remove_gases)]
df16_no_gas = df16[~df16["SUBSTANCE NAME"].isin(remove_gases)]

In [14]:
#drop rows where quantity released is 0 or "Below Reporting Threshold"
df_no_gas_cleaned = df_no_gas[~df_no_gas["QUANTITY RELEASED (kg)"].isin([0, "Below Reporting Threshold"])]
df_no_gas_cleaned["ROUTE NAME"] = df_no_gas_cleaned["ROUTE NAME"].str.strip().str.lower()
df_no_gas_cleaned["REGULATED INDUSTRY SECTOR"] = df_no_gas_cleaned["REGULATED INDUSTRY SECTOR"].str.strip().str.lower()


#reset index after dropping rows
df_no_gas_cleaned.reset_index(drop=True, inplace=True)
df_no_gas_cleaned.head()

C:\Users\andre\AppData\Local\Temp\ipykernel_24268\4124804988.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_gas_cleaned["ROUTE NAME"] = df_no_gas_cleaned["ROUTE NAME"].str.strip().str.lower()
C:\Users\andre\AppData\Local\Temp\ipykernel_24268\4124804988.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_gas_cleaned["REGULATED INDUSTRY SECTOR"] = df_no_gas_cleaned["REGULATED INDUSTRY SECTOR"].str.strip().str.lower()


,AUTHORISATION ID / PERMIT ID,ACTIVITY DESCRIPTION,OPERATOR NAME,SITE ADDRESS,SITE POSTCODE,EASTING,NORTHING,EA AREA NAME,ROUTE NAME,SUBSTANCE NAME,REPORTING THRESHOLD (kg),QUANTITY RELEASED (kg),REGULATED INDUSTRY SECTOR,REGULATED INDUSTRY SUB SECTOR,EA AREA NAME LABEL,ROUTE NAME LABEL,REGULATED INDUSTRY SECTOR LABEL,REGULATED INDUSTRY SUB SECTOR LABEL
0,BT5873IU,"INTENSIVE FARMING; > 40,000 POULTRY",LJ FAIRBURN AND SON LTD,Ivy House Farm Poultry Unit Ivy House Farm Far...,LN13 9PL,546500.0,374790.0,Lincolnshire and Northamptonshire,air,Particulate matter - PM10,1000.0,374,agriculture,Intensive Farming,7,0,0,11
1,XP3539RM,"INTENSIVE FARMING; > 40,000 POULTRY",Ridgeway Foods Limited,Seisdon Poultry Unit Fox Road Shipley West Mid...,WV6 7EL,382570.0,295320.0,West Midlands,air,Particulate matter - PM10,1000.0,395,agriculture,Intensive Farming,12,0,0,11
2,TP3636NV,"INTENSIVE FARMING; > 40,000 POULTRY",Fridays Ltd,Bowlands Farm Poultry Unit Tenterden Road Bidd...,TN27 8BH,585550.0,137170.0,"Kent, South London and East Sussex",air,Particulate matter - PM10,1000.0,618,agriculture,Intensive Farming,6,0,0,11
3,PP3030QE,"INTENSIVE FARMING; > 40,000 POULTRY",Hill Crest Eggs Limited,Cumberworth Road Cumberworth Lincolnshire,LN13 9LB,550370.0,374090.0,Lincolnshire and Northamptonshire,air,Particulate matter - PM10,1000.0,689,agriculture,Intensive Farming,7,0,0,11
4,HP3634CZ,"INTENSIVE FARMING; > 40,000 POULTRY",Watkins; Watkins,Stonehouse Farm Kempley Dymock Gloucestershire,GL18 2AT,367380.0,231840.0,West Midlands,air,Particulate matter - PM10,1000.0,800,agriculture,Intensive Farming,12,0,0,11


In [29]:
print(df.columns)
#remove air substances
df_no_gas_cleaned = df_no_gas_cleaned[df_no_gas_cleaned["ROUTE NAME"] != "air"]

# Assume you've already defined the dataframes:
df_dict = {
    "df_no_gas": df_no_gas,
    "df22_no_gas": df22_no_gas,
    "df21_no_gas": df21_no_gas,
    "df20_no_gas": df20_no_gas,
    "df19_no_gas": df19_no_gas,
    "df18_no_gas": df18_no_gas,
    "df17_no_gas": df17_no_gas,
    "df16_no_gas": df16_no_gas
}

# Apply the filter to each dataframe in the dictionary
for name, df in df_dict.items():
    df_dict[name] = df[df["ROUTE NAME"] != "air"]

Index(['AUTHORISATION ID / PERMIT ID', 'PRIMARY INDUSTRY ACTIVITY DESCRIPTION',
       'OPERATOR NAME', 'SITE ADDRESS', 'SITE POSTCODE', 'EASTING', 'NORTHING',
       'EA AREA NAME', 'ROUTE NAME', 'SUBSTANCE NAME',
       'REPORTING THRESHOLD (kg)', 'QUANTITY RELEASED (kg)',
       'REGULATED INDUSTRY SECTOR', 'REGULATED INDUSTRY SUB SECTOR'],
      dtype='object')


In [31]:
# Step 1: Filter each DataFrame to only include rows where SUBSTANCE NAME is 'copper'
for name, df in df_dict.items():
    df = df[df["SUBSTANCE NAME"] == "Copper"]
    
    # Step 3: Update the dictionary with the transformed DataFrame
    df_dict[name] = df

df_no_gas = df_dict["df_no_gas"]
df22_no_gas = df_dict["df22_no_gas"]
df21_no_gas = df_dict["df21_no_gas"]
df20_no_gas = df_dict["df20_no_gas"]
df19_no_gas = df_dict["df19_no_gas"]
df18_no_gas = df_dict["df18_no_gas"]
df17_no_gas = df_dict["df17_no_gas"]
df16_no_gas = df_dict["df16_no_gas"]

In [32]:
df21_no_gas.head()

,AUTHORISATION ID / PERMIT ID,ACTIVITY DESCRIPTION,OPERATOR NAME,SITE ADDRESS,SITE POSTCODE,EASTING,NORTHING,EA AREA NAME,ROUTE NAME,SUBSTANCE NAME,REPORTING THRESHOLD (kg),QUANTITY RELEASED (kg),REGULATED INDUSTRY SECTOR,REGULATED INDUSTRY SUB SECTOR
33,BS3000ID,ORGANIC CHEMICALS; SULPHUR CONTAINING COMPOUND...,STEPAN UK LTD,Stalybridge Organic Chemicals Bridge Street Ch...,SK15 1PH,395790.0,398370.0,Greater Manchester Merseyside and Cheshire,Wastewater,Copper,20.0,Below Reporting Threshold,Chemicals,Chemicals
122,FP3034PC,SURFACE TREATING METALS AND PLASTICS; ELECTROL...,The Diamond Metal Finishing Company Limited,"Unit 6,7A Newfield Industrial Estate High Stre...",ST6 5PB,385750.0,352250.0,West Midlands,Wastewater,Copper,20.0,Below Reporting Threshold,Metals,Surface treatment
140,LP3030FP,SURFACE TREATING METALS AND PLASTICS; ELECTROL...,Comer Plating Limited,Alpha Business Park Deedmore Road Coventry Wes...,CV2 1EQ,436900.0,282400.0,West Midlands,Wastewater,Copper,20.0,Below Reporting Threshold,Metals,Surface treatment
153,LP3933LX,COMBUSTION; ANY FUEL =>50MW,Chrysaor Production (U.K.) Limited,Theddlethorpe Gas Terminal Mabelthorpe Road Th...,LN12 1NQ,548690.0,387390.0,Lincolnshire and Northamptonshire,Controlled Waters,Copper,20.0,Below Reporting Threshold,Refineries & Fuel,Refineries & Fuel
174,VP3132FV,INORGANIC CHEMICALS; USING HALOGENS ETC IF REL...,RTI Advanced Forming Limited,Chemical Machining of Titanium Watchmead Hertf...,AL7 1LT,525190.0,212780.0,Hertfordshire and North London,Air,Copper,10.0,Below Reporting Threshold,Chemicals,Chemicals


In [33]:
#show what each label represents
for col in categorical_columns:
    unique_values = {value: idx for idx, value in enumerate(sorted(df_no_gas_cleaned[col].unique()))}
    print(f"\nLabel mappings for {col}:")
    for value, label in unique_values.items():
        print(f"{label}: {value}")


Label mappings for EA AREA NAME:
0: Cumbria and Lancashire
1: Devon, Cornwall and the Isles of Scilly
2: East Anglia
3: East Midlands
4: Greater Manchester Merseyside and Cheshire
5: Hertfordshire and North London
6: Kent, South London and East Sussex
7: Lincolnshire and Northamptonshire
8: North East
9: Solent and South Downs
10: Thames
11: Wessex
12: West Midlands
13: Yorkshire

Label mappings for ROUTE NAME:
0: controlled waters
1: wastewater

Label mappings for REGULATED INDUSTRY SECTOR:
0: biowaste treatment
1: cement and minerals
2: chemicals
3: combustion
4: efw
5: food & drink
6: landfill
7: metals
8: nuclear
9: paper and textiles
10: refineries & fuel
11: waste treatment
12: water industry

Label mappings for REGULATED INDUSTRY SUB SECTOR:
0: Biowaste Treatment
1: Cement
2: Chemicals
3: EfW
4: Ferrous
5: Food & Drink
6: Hazardous Waste
7: Inert Landfill
8: Metals Recycling
9: Non Hazardous Landfill
10: Non-Ferrous
11: Non-Hazardous & Inert
12: Nuclear
13: Paper & Pulp
14: Pow

In [34]:
#convert Easting/Northing to Latitude/Longitude
in_proj = Proj(init='epsg:27700')  #British National Grid
out_proj = Proj(init='epsg:4326')  #WGS84 (Latitude/Longitude)

#CONVERSION
def convert_coordinates(row):
    lon, lat = transform(in_proj, out_proj, row['EASTING'], row['NORTHING'])
    return pd.Series([lat, lon])

#df_no_gas_cleaned[['Latitude', 'Longitude']] = df_no_gas_cleaned.apply(convert_coordinates, axis=1)

C:\Users\andre\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\andre\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [35]:
df_no_gas[['Latitude', 'Longitude']] = df_no_gas.apply(convert_coordinates, axis=1)


C:\Users\andre\AppData\Local\Temp\ipykernel_24268\3885488271.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(in_proj, out_proj, row['EASTING'], row['NORTHING'])


In [36]:
df22_no_gas[['Latitude', 'Longitude']] = df22_no_gas.apply(convert_coordinates, axis=1)
df21_no_gas[['Latitude', 'Longitude']] = df21_no_gas.apply(convert_coordinates, axis=1)
df20_no_gas[['Latitude', 'Longitude']] = df20_no_gas.apply(convert_coordinates, axis=1)
df19_no_gas[['Latitude', 'Longitude']] = df19_no_gas.apply(convert_coordinates, axis=1)
df18_no_gas[['Latitude', 'Longitude']] = df18_no_gas.apply(convert_coordinates, axis=1)
df17_no_gas[['Latitude', 'Longitude']] = df17_no_gas.apply(convert_coordinates, axis=1)
df16_no_gas[['Latitude', 'Longitude']] = df16_no_gas.apply(convert_coordinates, axis=1)


C:\Users\andre\AppData\Local\Temp\ipykernel_24268\3885488271.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(in_proj, out_proj, row['EASTING'], row['NORTHING'])
C:\Users\andre\AppData\Local\Temp\ipykernel_24268\3885488271.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(in_proj, out_proj, row['EASTING'], row['NORTHING'])
C:\Users\andre\AppData\Local\Temp\ipykernel_24268\3885488271.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(in_proj, out_proj, row['EASTING'], row['NORTHING'])
C:\Users\andre\AppData\Local\Temp\ipykernel_24268\3885488271.py:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotc

In [47]:
df_no_gas.head()


,AUTHORISATION ID / PERMIT ID,ACTIVITY DESCRIPTION,OPERATOR NAME,SITE ADDRESS,SITE POSTCODE,EASTING,NORTHING,EA AREA NAME,ROUTE NAME,SUBSTANCE NAME,REPORTING THRESHOLD (kg),QUANTITY RELEASED (kg),REGULATED INDUSTRY SECTOR,REGULATED INDUSTRY SUB SECTOR,EA AREA NAME LABEL,ROUTE NAME LABEL,REGULATED INDUSTRY SECTOR LABEL,REGULATED INDUSTRY SUB SECTOR LABEL,Latitude,Longitude
3001,KP3836LT,DISPOSAL OF > 50 T/D NON-HAZARDOUS WASTE (> 10...,Yorkshire Water Services Limited,South Elmsall Wwtw Chapel Lane Pontefract West...,WF9 2SW,448170.0,410750.0,Yorkshire,Controlled Waters,Copper,20.0,31.09,Biowaste Treatment,Biowaste Treatment,13,1,1,0,53.591124,-1.273737
3007,LP3439LK,DISPOSAL OF > 50 T/D NON-HAZARDOUS WASTE (> 10...,NORTHUMBRIAN WATER LTD,"Industrial Effluent Treatment Works,Bran Sands...",TS6 6UE,456210.0,524290.0,North East,Controlled Waters,Copper,20.0,61.335,Biowaste Treatment,Biowaste Treatment,8,1,1,0,54.610652,-1.131224
3130,XP3705MV,Disposal of non-hazardous waste in a facility ...,Syracuse Waste Ltd,Maple Lodge WWTW Denham Way Hertfordshire,WD3 9SQ,503860.0,192210.0,Hertfordshire and North London,Wastewater,Copper,20.0,Below Reporting Threshold,Biowaste Treatment,Biowaste Treatment,5,3,1,0,51.619141,-0.501220
3140,EP3396SX,RECOVERY OR A MIX OF RECOVERY AND DISPOSAL OF ...,Lancashire Renewables Limited,Thornton Waste Treatment Facility Fleetwood Ro...,FY5 4PX,333700.0,444600.0,Cumbria and Lancashire,Wastewater,Copper,20.0,Below Reporting Threshold,Biowaste Treatment,Biowaste Treatment,0,3,1,0,53.893318,-3.010342
3151,EP3397EA,RECOVERY OR A MIX OF RECOVERY AND DISPOSAL OF ...,Lancashire Renewables Limited,Leyland Waste Treatment Facility Off Enterpris...,PR26 6TB,353740.0,423790.0,Cumbria and Lancashire,Wastewater,Copper,20.0,Below Reporting Threshold,Biowaste Treatment,Biowaste Treatment,0,3,1,0,53.708472,-2.702300


In [ ]:
df18_no_gas.shape[0]


In [48]:
import json
from shapely.geometry import shape, Point
import geopandas as gpd

# Your existing dataframe
# Make sure it has columns: LATITUDE and LONGITUDE
# Example: df_no_gas_cleaned = pd.read_csv("your_data.csv")

# Load the GeoJSON manually (replace with your actual path)
with open("Regions_December_2024_Boundaries_EN_BFC_1195854647342073399 (1).geojson") as f:
    regions_geojson = json.load(f)

# Extract region polygons and names
region_shapes = []
for feature in regions_geojson["features"]:
    region_name = feature["properties"]["RGN24NM"]  # Adjust this to match your GeoJSON field
    polygon = shape(feature["geometry"])
    region_shapes.append((region_name, polygon))

# Function to find which region a point belongs to
def get_region_for_point(lat, lon):
    point = Point(lon, lat)
    for name, polygon in region_shapes:
        if polygon.contains(point):
            return name
    return None  # If it doesn't fall into any known region

# Apply function to your dataframe
#df_no_gas_cleaned["REGION"] = df_no_gas_cleaned.apply(
 #   lambda row: get_region_for_point(row["Latitude"], row["Longitude"]),
  #  axis=1
#)

# Apply function to your dataframe
df_no_gas["REGION"] = df_no_gas.apply(
    lambda row: get_region_for_point(row["Latitude"], row["Longitude"]),
    axis=1
)
# Check result
#print(df_no_gas_cleaned[["Latitude", "Longitude", "REGION"]].head())
print(df_no_gas[["Latitude", "Longitude", "REGION"]].head())

       Latitude  Longitude                    REGION
3001  53.591124  -1.273737  Yorkshire and The Humber
3007  54.610652  -1.131224                North East
3130  51.619141  -0.501220           East of England
3140  53.893318  -3.010342                North West
3151  53.708472  -2.702300                North West


In [49]:
df22_no_gas["REGION"] = df22_no_gas.apply(lambda row: get_region_for_point(row["Latitude"], row["Longitude"]), axis=1)
df21_no_gas["REGION"] = df21_no_gas.apply(lambda row: get_region_for_point(row["Latitude"], row["Longitude"]), axis=1)
df20_no_gas["REGION"] = df20_no_gas.apply(lambda row: get_region_for_point(row["Latitude"], row["Longitude"]), axis=1)
df19_no_gas["REGION"] = df19_no_gas.apply(lambda row: get_region_for_point(row["Latitude"], row["Longitude"]), axis=1)
df18_no_gas["REGION"] = df18_no_gas.apply(lambda row: get_region_for_point(row["Latitude"], row["Longitude"]), axis=1)
df17_no_gas["REGION"] = df17_no_gas.apply(lambda row: get_region_for_point(row["Latitude"], row["Longitude"]), axis=1)
df16_no_gas["REGION"] = df16_no_gas.apply(lambda row: get_region_for_point(row["Latitude"], row["Longitude"]), axis=1)


In [10]:
# Inspect one feature to see available keys
print(regions_geojson["features"][0]["properties"])


{'FID': 1, 'RGN24CD': 'E12000001', 'RGN24NM': 'North East', 'BNG_E': 417315, 'BNG_N': 600357, 'LONG': -1.72887, 'LAT': 55.29702, 'GlobalID': '4885458b-5614-4e44-87c5-26d5755e24c1'}


In [11]:
#df_no_gas_cleaned['REGION']

1891    East of England
1892    East of England
1893    East of England
1895    East of England
1896    East of England
             ...       
5982         North West
5983         North West
5984      West Midlands
5985         North West
5986             London
Name: REGION, Length: 2995, dtype: object

In [50]:
#df_no_gas_cleaned.reset_index(drop=True, inplace=True)

#df_no_gas_cleaned
df21_no_gas

,AUTHORISATION ID / PERMIT ID,ACTIVITY DESCRIPTION,OPERATOR NAME,SITE ADDRESS,SITE POSTCODE,EASTING,NORTHING,EA AREA NAME,ROUTE NAME,SUBSTANCE NAME,REPORTING THRESHOLD (kg),QUANTITY RELEASED (kg),REGULATED INDUSTRY SECTOR,REGULATED INDUSTRY SUB SECTOR,Latitude,Longitude,REGION
33,BS3000ID,ORGANIC CHEMICALS; SULPHUR CONTAINING COMPOUND...,STEPAN UK LTD,Stalybridge Organic Chemicals Bridge Street Ch...,SK15 1PH,395790.0,398370.0,Greater Manchester Merseyside and Cheshire,Wastewater,Copper,20.0,Below Reporting Threshold,Chemicals,Chemicals,53.482033,-2.064908,North West
122,FP3034PC,SURFACE TREATING METALS AND PLASTICS; ELECTROL...,The Diamond Metal Finishing Company Limited,"Unit 6,7A Newfield Industrial Estate High Stre...",ST6 5PB,385750.0,352250.0,West Midlands,Wastewater,Copper,20.0,Below Reporting Threshold,Metals,Surface treatment,53.067297,-2.214113,West Midlands
140,LP3030FP,SURFACE TREATING METALS AND PLASTICS; ELECTROL...,Comer Plating Limited,Alpha Business Park Deedmore Road Coventry Wes...,CV2 1EQ,436900.0,282400.0,West Midlands,Wastewater,Copper,20.0,Below Reporting Threshold,Metals,Surface treatment,52.438332,-1.458618,West Midlands
153,LP3933LX,COMBUSTION; ANY FUEL =>50MW,Chrysaor Production (U.K.) Limited,Theddlethorpe Gas Terminal Mabelthorpe Road Th...,LN12 1NQ,548690.0,387390.0,Lincolnshire and Northamptonshire,Controlled Waters,Copper,20.0,Below Reporting Threshold,Refineries & Fuel,Refineries & Fuel,53.362473,0.232994,East Midlands
174,VP3132FV,INORGANIC CHEMICALS; USING HALOGENS ETC IF REL...,RTI Advanced Forming Limited,Chemical Machining of Titanium Watchmead Hertf...,AL7 1LT,525190.0,212780.0,Hertfordshire and North London,Air,Copper,10.0,Below Reporting Threshold,Chemicals,Chemicals,51.799661,-0.185895,East of England
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31351,AP3138CM,RECOVERY OR A MIX OF RECOVERY AND DISPOSAL OF ...,Urbaser Environmental Limited,Courtauld Road Waste Treatment Facility Courta...,SS3 1FL,574250.0,190810.0,East Anglia,Wastewater,Copper,20.0,1.1,Biowaste Treatment,Biowaste Treatment,51.589197,0.513973,East of England
31768,BK7382IS,FERROUS METALS; APPLYING PROTECTIVE FUSED META...,Bridon International Ltd.,"Doncaster Wire Mill,CARR HILL BALBY SOUTH YORK...",DN4 8DG,457350.0,401300.0,East Midlands,Controlled Waters,Copper,20.0,0.2,Metals,Surface treatment,53.505272,-1.136794,Yorkshire and The Humber
31973,CP3031SX,THE INCINERATION OF NON-HAZARDOUS WASTE IN AN ...,Slough Heat & Power Limited,"Slough Heat and Power,Edinburgh Avenue 342 Edi...",SL1 4TU,495460.0,181440.0,Thames,Wastewater,Copper,20.0,0.028,EfW,EfW,51.523818,-0.625434,South East
32172,BL8317IK,NON-FERROUS METALS; MELTING WITH CAPACITY >4T/...,H J Enthoven Limited,British Lead Mills PEARTREE LANE HERTFORDSHIRE,AL7 3UB,524500.0,212900.0,Hertfordshire and North London,Air,Copper,10.0,0.00057,Metals,Non-Ferrous,51.800894,-0.195853,East of England


In [55]:
#create new excel files
#df_no_gas.to_excel("pollution_no_gas.xlsx",index=False)
#df_no_gas_cleaned.to_excel("pollution_no_gas_cleaned.xlsx",index=False)

df_no_gas.to_excel("pollution23_copper.xlsx", index=False)
df22_no_gas.to_excel("pollution22_copper.xlsx", index=False)
df21_no_gas.to_excel("pollution21_copper.xlsx", index=False)
df20_no_gas.to_excel("pollution20_copper.xlsx", index=False)
df19_no_gas.to_excel("pollution19_copper.xlsx", index=False)
df18_no_gas.to_excel("pollution18_copper.xlsx", index=False)
df17_no_gas.to_excel("pollution17_copper.xlsx", index=False)
df16_no_gas.to_excel("pollution16_copper.xlsx", index=False)


In [54]:
# Filter rows with missing REGION
missing_regions = df21_no_gas[df21_no_gas["REGION"].isna()]
print(missing_regions)

# Print index (row number) and SITE POSTCODE
#for index, row in missing_regions.iterrows():
 #   print(f"Row {index}: Longitude = {row['Longitude']},Latitude = {row['Latitude']}")



      AUTHORISATION ID / PERMIT ID  \
5713                    AENTS13068   
9100                      XP3032MQ   
9461                      GP3037SJ   
9936                      UP3832MC   
17797                     HP3031LM   
18647                     GP3498HL   
20083                     YP3236LF   
22519                     BN2832IK   
29283                 S/06/55232/R   
29636                    017160064   
29728                     AW3TS519   
29851                    CNTW.0890   
29876                   AEECS12290   

                                    ACTIVITY DESCRIPTION  \
5713                                                 NaN   
9100   DISPOSAL OF > 50 T/D NON-HAZARDOUS WASTE (> 10...   
9461   WASTE LANDFILLING; >10 T/D WITH CAPACITY >25,0...   
9936   DISPOSAL OF > 50 T/D NON-HAZARDOUS WASTE (> 10...   
17797  OTHER WASTE DISPOSAL; NON-HAZARDOUS WASTE >50T...   
18647  DISPOSAL OR RECOVERY OF HAZARDOUS WASTE WITH A...   
20083           OTHER WASTE DISPOSAL; WASTE OIL

In [ ]:
#def assign_region(lat, lon):
 #   point = Point(lon, lat)
  #  for region_name, polygon in region_shapes:
   #     if polygon.contains(point) or polygon.touches(point):
    #        return region_name
     #   if polygon.buffer(0.0001).contains(point):  # Handle edge cases
      #      return region_name
    #return None

# First get just the unmatched points
#missing_mask = df_no_gas_cleaned["REGION"].isna()

# Reassign REGION using the new robust method
#df_no_gas_cleaned.loc[missing_mask, "REGION"] = df_no_gas_cleaned[missing_mask].apply(
 #   lambda row: assign_region(row["Latitude"], row["Longitude"]), axis=1
#)

# Check if any are still unmatched
#still_missing = df_no_gas_cleaned[df_no_gas_cleaned["REGION"].isna()]
#print(f"Still missing after retry: {len(still_missing)}")
